# Basic Spark 

In the lecture we discussed -- now we'll try to actually use the framework for some basic operations. 

In particular, this notebook will walk you through some of the basic [Spark RDD methods](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD). As you'll see, there is a lot more to it than `map` and `reduce`.

We will explore the concept of "lineage" in Spark RDDs and construct some simple key-value pair RDDs to write our first Spark applications.

If you need a reminder of some of the python concepts discussed earlier, you can make use of the [python refresher notebook](python-refresher.ipynb).

## Starting up the Spark runtime: initializing a `SparkContext` 

The `SparkContext` provides you with the means of communicating with a Spark cluster. The Spark cluster in turn is controlled by a master which orchestrates pieces of work between the various executors. Every interaction with the Spark runtime happens through the `SparkContext` in one way or another. Creating a `SparkContext` is therefore the very first step that needs to happen before we do anything else. 

In [1]:
import getpass
import pyspark
conf = pyspark.conf.SparkConf()
conf.setMaster('local[2]')
conf.setAppName('spark-intro-{0}'.format(getpass.getuser()))
sc = pyspark.SparkContext.getOrCreate(conf)
conf = sc.getConf()
sc

<SparkContext master=local[2] appName=spark-intro-mareite>

Hurrah! We have a Spark Context! Now lets get some data into the Spark universe.

## Creating an RDD

The basic object you will be working with is the Spark data abstraction called a Resilient Distributed Dataset (RDD). This class provides you with methods to execute work on your data using the Spark cluster. The simplest way of creating an RDD is by using `parallelize` to distribute an array of data among the executors:

In [2]:
data = range(100)
data_rdd = sc.parallelize(data)
print('Number of elements: ', data_rdd.count())
print('Sum and mean: ', data_rdd.sum(), data_rdd.mean())

Number of elements:  100
Sum and mean:  4950 49.5


This computation was executed on two executors, which you can see by inspecting the Spark application user interface. Each Spark application runs its own dedicated Web UI -- right-click (or command-click on Mac) the `Spark UI` link two cells above to get to the UI. 

This gives you a lot of nice information about the state of your job, including stats on execution time of individual tasks, available memory on all of the executors, links to logs, etc. You will probably begin to appreciate some of this information more when things start to go wrong...

## Map/Reduce 

Lets bring some of the simple python-only examples from the [first notebook]('../intro/Spark_workshop_Introduction.ipynb) into the Spark framework. The first map function we made was simply doubling the input array, so lets do this here. 

Write the function `double_the_number` and then use this function with the `map` method of `data_rdd` to yield `double_rdd`:

In [3]:
def double_the_number(x) : 
    return x*2

In [4]:
help(data_rdd.map)

Help on method map in module pyspark.rdd:

map(f, preservesPartitioning=False) method of pyspark.rdd.PipelinedRDD instance
    Return a new RDD by applying a function to each element of this RDD.
    
    >>> rdd = sc.parallelize(["b", "a", "c"])
    >>> sorted(rdd.map(lambda x: (x, 1)).collect())
    [('a', 1), ('b', 1), ('c', 1)]



In [5]:
double_rdd = data_rdd.map(double_the_number)

Not much happened here - or at least, no tasks were launched (you can check the console and the Web UI). Spark simply recorded that the `data_rdd` maps into `double_rdd` via the `map` method using the `double_the_number` function. You can see some of this information by inspecting the RDD debug string: 

In [6]:
print(double_rdd.toDebugString().decode())

(2) PythonRDD[5] at RDD at PythonRDD.scala:48 []
 |  ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:489 []


In [7]:
# comparing the first few elements of the original and mapped RDDs using take
print(data_rdd.take(10))
print(double_rdd.take(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


Now if you go over to check on the stages in the Spark UI you'll see that jobs were run to grab data from the RDD. In this case, a single task was run since all the numbers needed reside in one partition. Here we used `take` to extract a few RDD elements, a very very very convenient method for checking the data inside the RDD and debugging your map/reduce operations. 

Often, you will want to make sure that the function you define executes properly on the whole RDD. The most common way of forcing Spark to execute the mapping on all elements of the RDD is to invoke the `count` method: 

In [8]:
double_rdd.count()

100

If you now go back to the [stages page](http://localhost:4040/stages), you'll see that four tasks were run for this stage. 

In our initial example of using `map` in pure python code, we also used an inline lambda function. For such a simple construct like doubling the entire array, the lambda function is much neater than a separate function declaration. This works exactly the same way here.

Map the `data_rdd` to `double_lambda_rdd` by using a lambda function to multiply each element by 2: 

In [9]:
double_lambda_rdd = data_rdd.map( lambda x : double_the_number(x) )
print(double_lambda_rdd.take(10))

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


Finally, do a simple `reduce` step, adding up all the elements of `double_lambda_rdd`:

In [11]:
from operator import add
double_lambda_rdd.reduce(add)

9900

In [12]:
double_lambda_rdd.sum()

9900

(Spark RDDs actually have a `sum` method which accomplishes essentially the same thing)

## When things go wrong

Undoubtedly your code will sometimes (often?) fail. This can be particularly baffling in a complex system like Spark because of the many layers of abstraction involved. However, some errors can still be informative and in particular Python errors will usually propagate to the top. It might be hard to find them at first, so lets have a look.

In [13]:
def bad_function(x):
    """This refers to some variables that don't exist"""
    return x*woop    

This is the python stack trace that we would get if we just ran this without Spark:

In [14]:
bad_function(1)

NameError: name 'woop' is not defined

Very easy to see what the problem is - we have an undefined variable. Once this is used in a Spark RDD method, things get a bit more difficult to parse.

In [15]:
data_rdd.map(bad_function).count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 17, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/hdp/current/spark2-client/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/usr/hdp/current/spark2-client/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/hdp/current/spark2-client/python/pyspark/rdd.py", line 2423, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/hdp/current/spark2-client/python/pyspark/rdd.py", line 2423, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/hdp/current/spark2-client/python/pyspark/rdd

The cell above failed and gave us a long stacktrace. At first it doesn't look like it has anything to do with our underlying error, but if you scroll down a bit to the part that says

```
Py4JJavaError: An error occurred while calling...
```

You will see that this actually at least tells us the underlying problem: 

```
NameError: name 'woop' is not defined
```

You will also see there that it says something like `File "<ipython-input-4-f227c9dd67c6>", line 3, in bad_function` which tells us which cell the code came from and which function was called. This information will be critical when you try to debug your own functions.

## Filtering

A critical step in many analysis tasks is to filter down the input data. In Spark, this is another *transformation*, i.e. it takes an RDD and maps it to a new RDD via a filter function. The filter function needs to evaluate each element of the RDD to either `True` or `False`. 

Use `filter` with a lambda function to select all values less than 10: 

In [16]:
filtered_rdd = data_rdd.filter(lambda x : x<10)
filtered_rdd.count()

10

Of course we can now apply the `map` and double the `filtered_rdd` just as before: 

In [17]:
filtered_rdd.map(lambda x : double_the_number(x)).take(10)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

Note that each RDD transformation returns a new RDD instance to the caller -- for example:

In [18]:
data_rdd.filter(lambda x: x % 2)

PythonRDD[17] at RDD at PythonRDD.scala:48

You can therefore string together many transformations without creating a separate instance variable for each step. Our `filter` + `map` step can therefore be combined into one. Note that if we surround the operations with "( )" we can make the code more readable by placing each transformation on a separate line: 

In [19]:
composite = (data_rdd.filter(lambda x: x % 2)
                     .map(lambda x: x*2))

Again, if you now look at the Spark UI you'll see that nothing actually happened -- no job was trigerred. The `composite` RDD simply encodes the information needed to create it. 

If an action is executed that only requires a part of the RDD, only those parts will be computed. If we cache the RDD and only calculate a few of the elements, this will be made clear:

In [20]:
composite.cache()
composite.take(10)

[2, 6, 10, 14, 18, 22, 26, 30, 34, 38]

If you look at the **Storage** tab in the Spark UI you'll see that just a quarter of the RDD is cached. Now if we trigger the full calculation, this will increase to 100%:

In [21]:
composite.count()

50

## Key, value pair RDDs

`key`,`value` pair data is the "bread and butter" of map/reduce programming. Think of the `value` part as the meat of your data and the `key` part as some crucial metadata. For example, you might have time-series data for CO$_2$ concentration by geographic location: the `key` might be the coordinates or a time window, and `value` the CO$_2$ data itself. 

If your data can be expressed in this way, then the map/reduce computation model can be very convenient for pre-processing, cleaning, selecting, filtering, and finally analyzing your data. 

Spark offers a `keyBy` method that you can use to produce a key from your data. In practice this might not be useful often but we'll do it here just to make an example: 

In [22]:
# key the RDD by x modulo 5
keyed_rdd = data_rdd.keyBy(lambda x: x%5)

In [23]:
keyed_rdd.take(20)

[(0, 0),
 (1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (0, 5),
 (1, 6),
 (2, 7),
 (3, 8),
 (4, 9),
 (0, 10),
 (1, 11),
 (2, 12),
 (3, 13),
 (4, 14),
 (0, 15),
 (1, 16),
 (2, 17),
 (3, 18),
 (4, 19)]

This created keys with values 0-4 for each element of the RDD. We can now use the multitude of `key` transformations and actions that the Spark API offers. For example, we can revisit `reduce`, but this time do it by `key`: 

## `reduceByKey`

In [24]:
# use the add operator in the `reduceByKey` method
red_by_key = keyed_rdd.reduceByKey(add)
red_by_key.collect()

[(0, 950), (2, 990), (4, 1030), (1, 970), (3, 1010)]

Unlike the global `reduce`, the `reduceByKey` is a *transformation* --> it returns another RDD. Often, when we reduce by key, the dataset size is reduced enough that it is safe to pull it completely out of Spark and into the driver (i.e. this notebook). A useful way of doing this is to automatically convert it to python dictionary for subsequent processing with the `collectAsMap` method:

In [25]:
red_dict = red_by_key.collectAsMap()
red_dict

{0: 950, 1: 970, 2: 990, 3: 1010, 4: 1030}

In [26]:
# access by key
red_dict[0]

950

## `groupByKey`

If you want to collect the elements belonging to a key into a list in order to process them further, you can do this with `groupByKey`. Note that if you want to group the elements only to do a subsequent reduction, you are far better off using `reduceByKey`, because it does the reduction locally on each partition first before communicating the results to the other nodes. By contrast, `groupByKey` reshuffles the entire dataset because it has to group *all* the values for each key from all of the partitions. 

In [27]:
keyed_rdd.groupByKey().collect()

[(0, <pyspark.resultiterable.ResultIterable at 0x7f03b8b2bf28>),
 (2, <pyspark.resultiterable.ResultIterable at 0x7f03b8b2bf98>),
 (4, <pyspark.resultiterable.ResultIterable at 0x7f03b9740a90>),
 (1, <pyspark.resultiterable.ResultIterable at 0x7f03b8b2b080>),
 (3, <pyspark.resultiterable.ResultIterable at 0x7f03b8b2b4a8>)]

Note the ominous-looking `pyspark.resultiterable.Resultiterable`: this is exactly what it says, an iterable. You can turn it into a list or go through it in a loop. For example:

In [28]:
key, iterable = keyed_rdd.groupByKey().first()

In [29]:
list(iterable)

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]

## `sortBy`

Use the `sortBy` method of `red_by_key` to return a list sorted by the sums in descending order and print it out. 

In [37]:
sorted_red = red_by_key.sortBy(lambda x: x[1], ascending=False ).collect()
sorted_red

[(4, 1030), (3, 1010), (2, 990), (1, 970), (0, 950)]

In [38]:
assert(sorted_red == [(4, 1030), (3, 1010), (2, 990), (1, 970), (0, 950)])

This concludes the brief tour of the Spark runtime -- we can now shut down the `SparkContex` by calling `sc.stop()`. This removes your job from the Spark cluster and cleans up the memory and temporary files on disk. 

In [39]:
sc.stop()